In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow.keras.layers as tfl
import cv2

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [2]:
# model = tf.keras.models.Sequential([
#     tfl.Conv2D(32, kernel_size=3, activation="relu", kernel_initializer="he_normal", input_shape=(1000, 1000, 3)),
#     tfl.MaxPooling2D(pool_size=(2,2)),
#     tfl.Dropout(0.1),
#     tfl.Conv2D(64, kernel_size=3, activation="relu", kernel_initializer="he_normal"),
#     tfl.MaxPooling2D(pool_size=(2,2)),
#     tfl.Dropout(0.2),
#     tfl.Conv2D(128, kernel_size=3, activation="relu", kernel_initializer="he_normal"),
#     tfl.MaxPooling2D(pool_size=(2,2)),
#     tfl.Flatten(),
#     tfl.Dense(1024, activation="relu"),
#     tfl.Dense(256, activation="relu"),
#     tfl.Dense(4, activation="softmax")
# ])
# model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [3]:
trained_model = tf.keras.applications.inception_v3.InceptionV3(input_shape=(500, 500, 3), 
                                                               include_top = False, weights="imagenet")

for layer in trained_model.layers:
    layer.trainable = False

87910968/87910968 [==============================] - 3s 0us/step


In [4]:
trained_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 500, 500, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 249, 249, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 249, 249, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                      

In [5]:
final_layer = trained_model.get_layer("mixed7")
final_output= final_layer.output

In [6]:
x = tfl.Flatten()(final_output)
x = tfl.Dense(1024, activation="relu")(x)
x = tfl.Dropout(0.2)(x)
x = tfl.Dense(4, activation="softmax")(x)

model = tf.keras.Model(trained_model.input, x)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 500, 500, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 249, 249, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 249, 249, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                             

In [7]:
testing_alpha = cv2.imread(os.path.join("/kaggle/input/vehicle-type-recognition/Dataset", "Bus", "Image_10.jpg"))
testing_alpha.shape

(2592, 3872, 3)

In [8]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255, 
                                  rotation_range = 40,
                                  width_shift_range = 0.2,
                                  height_shift_range = 0.2,
                                  shear_range = 0.2,
                                  zoom_range = 0.2,
                                 )

train_generator = train_datagen.flow_from_directory("/kaggle/input/vehicle-type-recognition/Dataset",
                                                   batch_size=20,
                                                   class_mode="sparse",
                                                   target_size=(500, 500)
                                                   )


Found 400 images belonging to 4 classes.


In [9]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.fit(train_generator, epochs=5)

Epoch 1/5


/opt/conda/lib/python3.10/site-packages/PIL/Image.py:992: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


20/20 [==============================] - 279s 13s/step - loss: 23.9973 - accuracy: 0.5300
Epoch 2/5
20/20 [==============================] - 267s 13s/step - loss: 3.9664 - accuracy: 0.7500
Epoch 3/5
20/20 [==============================] - 267s 13s/step - loss: 1.6674 - accuracy: 0.8575
Epoch 4/5
20/20 [==============================] - 267s 13s/step - loss: 1.5367 - accuracy: 0.8650
Epoch 5/5
20/20 [==============================] - 268s 13s/step - loss: 0.9723 - accuracy: 0.8725
